# TSLib v2 - Example notebook for full pipeline

In [1]:
%pip install pytorch-forecasting

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from typing import Any, Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler
import torch
from torch.optim import Optimizer
from torch.utils.data import Dataset

from pytorch_forecasting.data.encoders import (
    EncoderNormalizer,
    NaNLabelEncoder,
    TorchNormalizer,
)

In [3]:
from pytorch_forecasting.data.timeseries import TimeSeries
from pytorch_forecasting.data.tslib_data_module import TslibDataModule
from pytorch_forecasting.models.timexer import TimeXer

In [4]:
num_series = 100
seq_length = 50
data_list = []
for i in range(num_series):
    x = np.arange(seq_length)
    y = np.sin(x / 5.0) + np.random.normal(scale=0.1, size=seq_length)
    category = i % 5
    static_value = np.random.rand()
    for t in range(seq_length - 1):
        data_list.append(
            {
                "series_id": i,
                "time_idx": t,
                "x": y[t],
                "y": y[t + 1],
                "category": category,
                "future_known_feature": np.cos(t / 10),
                "static_feature": static_value,
                "static_feature_cat": i % 3,
            }
        )
data_df = pd.DataFrame(data_list)
data_df.head()

,series_id,time_idx,x,y,category,future_known_feature,static_feature,static_feature_cat
0,0,0,-0.095660,0.172945,0,1.000000,0.462314,0
1,0,1,0.172945,0.485776,0,0.995004,0.462314,0
2,0,2,0.485776,0.630254,0,0.980067,0.462314,0
3,0,3,0.630254,0.907009,0,0.955336,0.462314,0
4,0,4,0.907009,0.814266,0,0.921061,0.462314,0


In [5]:
dataset = TimeSeries(
    data=data_df,
    time="time_idx",
    target="y",
    group=["series_id"],
    num=["x", "future_know_feature", "static_feature"],
    cat=["category", "static_feature_cat"],
    known=["future_known_feature"],
    unknown=["x", "category"],
    static=["static_feature", "static_feature_cat"],
)

C:\Users\prana\Desktop\code\pytorch-forecasting\pytorch_forecasting\data\timeseries\_timeseries_v2.py:106: UserWarning: TimeSeries is part of an experimental rework of the pytorch-forecasting data layer, scheduled for release with v2.0.0. The API is not stable and may change without prior warning. For beta testing, but not for stable production use. Feedback and suggestions are very welcome in pytorch-forecasting issue 1736, https://github.com/sktime/pytorch-forecasting/issues/1736
  warn(


In [6]:
data_module = TslibDataModule(
    time_series_dataset=dataset,
    context_length=30,
    prediction_length=1,
    add_relative_time_idx=True,
    target_normalizer=TorchNormalizer(),
    categorical_encoders={
        "category": NaNLabelEncoder(add_nan=True),
        "static_feature_cat": NaNLabelEncoder(add_nan=True),
    },
    scalers={
        "x": StandardScaler(),
        "future_known_feature": StandardScaler(),
        "static_feature": StandardScaler(),
    },
    batch_size=32,
)

C:\Users\prana\Desktop\code\pytorch-forecasting\pytorch_forecasting\data\tslib_data_module.py:264: UserWarning: TslibDataModule is experimental and subject to change. The API is not stable and may change without prior warning.
  warnings.warn(


In [7]:
type(data_module.metadata)

dict

In [8]:
data_module.metadata

{'feature_names': {'categorical': ['category', 'static_feature_cat'],
  'continuous': ['x', 'future_known_feature', 'static_feature'],
  'static': ['static_feature', 'static_feature_cat'],
  'known': ['future_known_feature'],
  'unknown': ['x', 'category', 'static_feature', 'static_feature_cat'],
  'target': ['y'],
  'all': ['x',
   'category',
   'future_known_feature',
   'static_feature',
   'static_feature_cat'],
  'static_categorical': ['static_feature_cat'],
  'static_continuous': ['static_feature']},
 'feature_indices': {'categorical': [1, 4],
  'continuous': [0, 2, 3],
  'static': [],
  'known': [2],
  'unknown': [0, 1, 3, 4],
  'target': [0],
  'all': []},
 'n_features': {'categorical': 2,
  'continuous': 3,
  'static': 2,
  'known': 1,
  'unknown': 4,
  'target': 1,
  'all': 5,
  'static_categorical': 1,
  'static_continuous': 1},
 'context_length': 30,
 'prediction_length': 1,
 'freq': 'h',
 'features': 'MS'}

In [9]:
import torch.nn as nn

from pytorch_forecasting.metrics import MAE, SMAPE

In [10]:
model = TimeXer(
    loss=nn.MSELoss(),
    features="MS",
    d_model=64,
    nhead=4,
    e_layers=2,
    d_ff=256,
    dropout=0.1,
    patch_length=4,
    logging_metrics=[MAE(), SMAPE()],
    optimizer="adam",
    optimizer_params={"lr": 1e-3},
    lr_scheduler="reduce_lr_on_plateau",
    lr_scheduler_params={
        "mode": "min",
        "factor": 0.5,
        "patience": 5,
    },
    metadata=data_module.metadata,
)

C:\Users\prana\Desktop\code\pytorch-forecasting\pytorch_forecasting\models\base\_base_model_v2.py:60: UserWarning: The Model 'TimeXer' is part of an experimental reworkof the pytorch-forecasting model layer, scheduled for release with v2.0.0. The API is not stable and may change without prior warning. This class is intended for beta testing and as a basic skeleton, but not for stable production use. Feedback and suggestions are very welcome in pytorch-forecasting issue 1736, https://github.com/sktime/pytorch-forecasting/issues/1736
  warn(


In [11]:
from lightning.pytorch import Trainer

trainer = Trainer(
    max_epochs=5,
    accelerator="auto",
    devices=1,
    enable_progress_bar=True,
    enable_model_summary=True,
)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(model, data_module)


  | Name         | Type                   | Params | Mode 
----------------------------------------------------------------
0 | loss         | MSELoss                | 0      | train
1 | en_embedding | EnEmbedding            | 320    | train
2 | ex_embedding | DataEmbedding_inverted | 2.0 K  | train
3 | encoder      | Encoder                | 133 K  | train
4 | head         | FlattenHead            | 513    | train
----------------------------------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.546     Total estimated model params size (MB)
57        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Pl

Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likel

...

c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likel

Validation: |          | 0/? [00:00<?, ?it/s]

c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likel

Validation: |          | 0/? [00:00<?, ?it/s]

c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likel

In [13]:
test_metrics = trainer.test(model, data_module)

c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\prana\Desktop\code\pytorch-forecasting\.venv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likel

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_MAE            0.37911707162857056
       test_SMAPE           0.8660855889320374
        test_loss           0.1897258311510086
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [14]:
model.eval()

TimeXer(
  (loss): MSELoss()
  (en_embedding): EnEmbedding(
    (value_embedding): Linear(in_features=4, out_features=64, bias=False)
    (position_embedding): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (ex_embedding): DataEmbedding_inverted(
    (value_embedding): Linear(in_features=30, out_features=64, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (layers): ModuleList(
      (0-1): 2 x EncoderLayer(
        (self_attention): AttentionLayer(
          (inner_attention): FullAttention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (query_projection): Linear(in_features=64, out_features=64, bias=True)
          (key_projection): Linear(in_features=64, out_features=64, bias=True)
          (value_projection): Linear(in_features=64, out_features=64, bias=True)
          (out_projection): Linear(in_features=64, out_features=64, bias=True)
        )
        (cross_attention): AttentionLayer(
 

In [15]:
with torch.no_grad():
    test_batch = next(iter(data_module.test_dataloader()))
    x_test, y_test = test_batch
    y_pred = model(x_test)

    print("Prediction:", y_pred["prediction"])

Prediction: tensor([[0.4459],
        [0.4574],
        [0.4523],
        [0.4688],
        [0.4577],
        [0.4392],
        [0.4170],
        [0.4361],
        [0.4367],
        [0.4475],
        [0.4552],
        [0.4350],
        [0.4449],
        [0.4299],
        [0.4112],
        [0.4330],
        [0.4180],
        [0.4473],
        [0.4113],
        [0.4923],
        [0.4416],
        [0.4613],
        [0.4616],
        [0.4454],
        [0.4567],
        [0.4397],
        [0.4370],
        [0.4406],
        [0.4291],
        [0.4336],
        [0.4317],
        [0.4308]])
